https://book.st-hakky.com/hakky/google-spreadsheet-read/
こちらを参考にして作成した

In [8]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
import os

worksheets = ["入力用(京阪神)","入力用(関西) ","入力用(全国) "]
Auth = "./API_KEY/local-snow-429706-d4-e028ed82a83f.json" # ここにjsonファイルのPathを入力する
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = Auth
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file(Auth, scopes=scope)
Client = gspread.authorize(credentials)

SpreadSheet = Client.open_by_key("1kP892zDoyfpohOZKooVUZmcdfX6Y0FebYOiJTGsS6jY") #スプレッドシートのidをここに入力(urlに書いてある)

SpreadSheet_Data = []
for sheets in worksheets:
    RawData = SpreadSheet.worksheet(sheets)
    Data = pd.DataFrame(RawData.get_all_values())
    SpreadSheet_Data.append(Data)

In [70]:
all_list_data = [] #workseetsのデータを全てlist形式に変換して保存します.順序はworksheetsの名前順
for i in range(3):
    Sheet = SpreadSheet_Data[i]
    dict_Sheet = Sheet.to_dict()
    pref = [v for k,v in dict_Sheet[3].items()][2:] #0,1は空白とdfの名前なので除外
    url  = [v for k,v in dict_Sheet[7].items()][2:]
    disc  = [v for k,v in dict_Sheet[8].items()][2:]
    museum = [v for k,v in dict_Sheet[2].items()][2:]
    exhibition = [v for k,v in dict_Sheet[4].items()][2:]
    start = [v for k,v in dict_Sheet[5].items()][2:]
    end = [v for k,v in dict_Sheet[6].items()][2:]
    all_data = [ [museum[i],pref[i],exhibition[i],start[i],end[i],url[i],disc[i]] for i in range(len(pref))]
    all_list_data.append(all_data)
print(all_list_data[0])

[['平等院ミュージアム鳳翔館', '京都府', '縁を結ぶ -浄土と源氏物語の情景-', '2024/10/16', '2025/1/13', 'https://www.byodoin.or.jp/museum/', 'この度、平等院ミュージアム鳳翔館では春季特別展「縁を結ぶ -浄土と源氏物語の情景-」展を開催します。平等院鳳凰堂が創建された平安時代、仏教は人の行動と結びつき、さらに自分自身の存在とも結びついていました。貴族たちの間では浄土信仰が盛んになりました。仏との縁を結んで経典の読誦などの仏事を行い、仏菩薩が浄土往生にいざなうさまは来迎図として描かれました。また、平安時代を代表する文学作品である『源氏物語』では、光源氏ら貴族たちの人生や女性たちとの出会いは、たびたび仏教にもとづく前世との関わりから生じると考えられました。本展では、浄土信仰と源氏物語に関連した、平安時代から江戸時代までの作品を中心に紹介します。'], ['◎アサヒグループ大山崎山荘美術館', '京都府', '丸沼芸術の森所蔵 アンドリュー・ワイエス展', '2024/9/14', '2024/12/8', 'https://www.asahigroup-oyamazaki.com/', 'アサヒグループ大山崎山荘美術館は、2024年9月14日(土)から12月8日(日)を2期に分けて、企画展「丸沼芸術の森所蔵 アンドリュー・ワイエス展 ―追憶のオルソン・ハウス」を開催いたします。アンドリュー・ワイエス(1917–2009)は、アメリカの国民的画家として知られます。ニューヨーク西方のペンシルベニア州に生まれ、生涯の多くの時間を故郷と、アメリカ最北東部のメイン州で過ごし、それらの風景とそこに生きる人々を描きました。とりわけ、《クリスティーナの世界》（1948年）は、20世紀のアメリカ美術を代表する傑作として高く評価されています。1939年、ワイエスは自身の別荘があるメイン州でクリスティーナ・オルソンと弟アルヴァロに出会います。ふたりと、彼らの住む築約150年の古い屋敷にひかれ、画家は30年にわたり同家を訪れ交流をもちました。この「オルソン・ハウス」を舞台に《クリスティーナの世界》をはじめとする数々の名作が描かれました。本展では、埼玉県朝霞市にある丸沼芸術の森が所蔵する、《クリスティーナの世界》習作を

In [71]:
all_kansai_list = []
other_japan_list = []
for i in range(2):
    for data in all_list_data[i]:
        if data[6]!="":all_kansai_list.append(data) #説明が記述されているのなら追加

for data in all_list_data[2]:
    if data[6]!="":other_japan_list.append(data) #説明が記述されているのなら追加

In [75]:
print(len(other_japan_list))
print(len(all_kansai_list))

387


In [74]:
# ベクトルデータベース
#関西を作成します
import openai
import os, json, re
from pprint import pprint

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY


index = []
for i, doc in enumerate(all_kansai_list):
 
    text = f"{doc[1]}にある「{doc[0]}」で行われる「{doc[2]}」は{doc[3]}から{doc[4]}まで開催されます。その内容は「{doc[6]}」"
    # ここでベクトル化を行う
    # openai.embeddings_utils.embeddings_utilsを使うともっとシンプルにかけます
    res = openai.Embedding.create(
        model='text-embedding-3-small',
        input=text
    )
    # ベクトルをデータベースに追加
    index.append({
        'number': i+1,
        'name': doc[2],
        'raw': doc,
        'embedding_text': text,
        'embedding': res['data'][0]['embedding']
    })

with open('embedding_kansai.json', 'w', encoding="utf8") as f:
    json.dump(index, f)

In [73]:
# ベクトルデータベース
#　上記で作成したindexに他日本の情報を追加することでall_japan_listを作成・保存します

for i, doc in enumerate(other_japan_list):
 
    text = f"{doc[1]}にある「{doc[0]}」で行われる「{doc[2]}」は{doc[3]}から{doc[4]}まで開催されます。その内容は「{doc[6]}」"
    # ここでベクトル化を行う
    # openai.embeddings_utils.embeddings_utilsを使うともっとシンプルにかけます
    res = openai.Embedding.create(
        model='text-embedding-3-small',
        input=text
    )
    # ベクトルをデータベースに追加
    index.append({
        'number': i+1,
        'name': doc[2],
        'raw': doc,
        'embedding_text': text,
        'embedding': res['data'][0]['embedding']
    })

with open('embedding_all.json', 'w', encoding="utf8") as f:
    json.dump(index, f)

In [69]:
#2023を振り返る
with open('./2023/embedding_all.json') as f:
    embedding_all = json.load(f)
all_data_2023 = [ d['raw']for d in embedding_all]

for d in all_data_2023:
    if d[2]=="常設展":print(d)

['名古屋港水族館', '愛知県', '常設展', '2023/1/1', '2023/12/31', 'http://www.nagoyaaqua.jp', '約35億年前、地球上初の生命体として海の中に単細胞生物が誕生しました。無数の生命がそれぞれ進化の途を辿り、陸地へと進出した生物の一部は巨大な恐竜へと進化して陸地を支配するようになりましたが、やがて絶滅していきました。その後、哺乳類たちが陸地の主役となり、約5,000万年前、哺乳類たちの一部が生命の故郷である海へと戻っていきました。彼らこそ、現代の鯨へと進化していく動物でした。彼らは、水中生活に適応するために体型を無駄のない流線型のフォルムへと変化させていき、現在私たちが見ている美しい姿へと進化していきました。悠久な生命進化の歴史の中で、水中生活に適応し素晴らしい知性を発達させ、陸上の人間の地位に匹敵するといわれる海洋の生活者である鯨類の世界を、順路に沿って紹介します。地球の約70%を占める広大な海は、水域によって水温や気候が異なるため、生き物たちは様々な環境に適応し、進化を続けてきました。南館では、名古屋港ガーデンふ頭に係留されている「南極観測船ふじ」がかつて南極へ向かった際のコースをたどり、「日本の海」「深海ギャラリー」「赤道の海」「オーストラリアの水辺」「南極の海」という様々な水域で暮らす生き物たちを観察することができます。南極への旅に出発し、様々な水域に暮らす生き物たちの生態を間近に観察してみましょう。', '48']
['彫刻の森美術館', '神奈川県', '常設展', '2023/1/1', '2023/12/31', 'https://www.hakone-oam.or.jp', '作品の多くは、耐久性のある恒久的な種類の彫刻に限定されています。野外彫刻が成立するには、風雪と時の流れに耐え、長く存在することにあると考えるからです。従って、作品が影響を持ち、人々に知られるようになるには長い時間を要します。また、館内にあるそれぞれの彫刻は、空間との兼ね合いを図って展示されています。背景の自然は四季折々に変化し、見る人を楽しませてくれます。作品は芸術家の言葉です。芸術家は自らの時代を生き、時代に訴え、時には時代を超えた真実を未来へ向けて語ろうとしています。その言葉に耳を傾け、対話してみてください。世界